# MS consumption

In [ ]:

import os
import time
import re

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import style
from scipy import stats


reffiles2000 = ['C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20160301\\20160301_conso2000.csv',
                'C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20160307\\20160307_conso2000.csv',
                'C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20160302\\20160302_conso2000.csv']
# reffiles2000 = ['C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20160301\\20160301_conso2000.csv',
#                 'C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20160204\\20160204_cons2000b.csv',
#                 'C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20151014\\20151014_calO2.csv',
#                'C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20151209\\20151209_oxygen_cons_1.csv',
#                'C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20151209\\20151209_oxygen_cons_2.csv',
#                'C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20151209\\20151209_oxygen_cons_4.csv',
#                'C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20151209\\20151209_oxygen_cons_6.csv',
#                'C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20160203\\20160203_Argon.csv']

reffiles600 = ['C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20160204\\20160204_cons600.csv',
               'C:\\Users\\u5040252\\My Documents\\Projects\\Mass_Spec_Data\\20151016\\\20151016_model_argon_600_1.csv']

def rolling_linear_reg( x, y):
        r = np.zeros(len(x))
        r[:] = np.NAN
        myrange = range(int(60 / 2), len(x) - int(60 / 2))

        a = np.vstack([x, np.ones(len(x))]).T
        for i in myrange:
            r[i] = np.linalg.lstsq(a[i:i + 60], y[i:i + 60])[0][0]
        return r
    
def consumption_params(reffile, limit=7200):
        df = pd.read_csv(reffile, encoding='ISO-8859-1')

        if 'time2' not in df.columns:
            df['time2'] = df['time']
            
        df = df[['time2', 'Mass32', 'Mass40']]
            
        if df.time2.max() > limit:
            df = df.loc[df.time2 < limit, :]

        df['d32dt'] = rolling_linear_reg(df.time2, df['Mass32'])
        df['d40dt'] = rolling_linear_reg(df.time2, df['Mass40'])

        # calculate slope d32dt = f(Mass32) and d40dt = f(Mass40)
        x32, y32 = np.array(df.loc[df.d32dt.notnull(), "Mass32"]), np.array(df.loc[df.d32dt.notnull(), "d32dt"])
        x40, y40 = np.array(df.loc[df.d40dt.notnull(), "Mass40"]), np.array(df.loc[df.d40dt.notnull(), "d40dt"])

        f32 = np.linalg.lstsq(x32[:, np.newaxis], y32)[0][0]
        f40 = np.linalg.lstsq(x40[:, np.newaxis], y40)[0][0]

#         with open(self.paramfile, "w") as f:
#             f.write("".join([str(f32), "\n", str(se.f40)]))
        return f32, f40


In [ ]:
# reference files for 600 uL cuvette
b1=time.time()

for f in reffiles2000:
    if os.path.isfile(f):
        b2=time.time()
        f32, f40 =  consumption_params(f)
        print(f.split("\\")[-1], f32, f40, time.time() - b2)
    else:
        print("couldn't find: " + f)
    
print(time.time() - b1)

In [ ]:
# reference files for 2000 uL cuvette
b1=time.time()


for f in reffiles600:
    b2=time.time()
    
    if os.path.isfile(f):
        f32, f40 =  consumption_params(f)
    print(f.split("\\")[-1], f32, f40, time.time() - b2)
    
    
print(time.time() - b1)